In [67]:
from sqlalchemy import create_engine
from tqdm import tqdm
import pymysql
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time

In [68]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()


def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn

def get_pymysql_stock_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result
               

def get_empty_day_df(sqlalchemy_conn):
     
    sql = "SELECT * FROM investing_data.aedkrw내역"
            
    result = sqlalchemy_conn.execute(sql)
    empty_day_df = pd.DataFrame(result.fetchall())
    
    empty_day_df = empty_day_df.set_index('날짜')
    empty_day_df = empty_day_df.notnull().replace(True, np.NaN)

    empty_day_df = empty_day_df.reset_index()
    empty_day_df = empty_day_df.drop(columns=['AEDKRW내역_종가','AEDKRW내역_오픈','AEDKRW내역_고가'
                                      ,'AEDKRW내역_저가','AEDKRW내역_거래량','AEDKRW내역_변동'])
    empty_day_df.to_pickle('./pickle/empty_day_df.pkl')
    
    return empty_day_df

def get_sqlalchemy_investing_data(day_df, conn, investing_table_list):
    
    investing_df = pd.DataFrame(day_df)
    for table in investing_table_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        
        sql = "SELECT * FROM investing_data.`{0}`".format(table)
        result = conn.execute(sql)
        table_df = pd.DataFrame(result.fetchall())
        # table_df = table_df.set_index('날짜')
        # df1.join(df2)
        investing_df = pd.merge(investing_df, table_df,on='날짜', how='left')
        
    for c in list(investing_df.columns):
        if c.split('_')[-1] == '거래량' or c.split('_')[-1] == '변동':
            investing_df[c] = investing_df[c].fillna(0)
        else:
            investing_df[c] = investing_df[c].fillna(method='bfill')
            
    return investing_df  
        
def get_sqlalchemy_stock_data(conn, stock_table_list, investing_df):
    
    stock_df = pd.DataFrame()
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) 
    for table in tqdm(stock_table_list[:10]):
        sql = "SELECT * FROM stock_info.`{0}`".format(table)
        print(table)
        result = conn.execute(sql)
        table_df = pd.DataFrame(result.fetchall())
        # if table_df.isnull().sum() == 0:
        table_df['날짜'] = table_df['날짜'].astype(str).astype(int)
        merge_df = pd.merge(table_df, investing_df, on='날짜', how='left')
        # merge_df = merge_df.fillna(method='bfill')

        stock_df = pd.concat([stock_df, merge_df], axis=0)
    
    
    conn.close()
    return stock_df     


In [69]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
investing_table_list = get_pymysql_stock_list(pymysql_conn, 'investing_data')
empty_day_df = get_empty_day_df(sqlalchemy_conn)

KeyboardInterrupt: 

In [ ]:
investing_df = get_sqlalchemy_investing_data(empty_day_df, sqlalchemy_conn, investing_table_list)

In [ ]:
investing_df

,날짜,AEDKRW내역_종가,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,AEDKRW내역_거래량,AEDKRW내역_변동,ATX내역_종가,ATX내역_오픈,ATX내역_고가,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,20200101,314.19,314.46,314.53,314.20,0,0.01,3229.37,3187.62,3239.10,...,46476.20,45879.12,0,0.00,6690.6,6684.1,6709.2,6669.2,0.0,0.00
1,20200102,315.05,314.47,316.12,314.03,0,0.27,3229.37,3187.62,3239.10,...,46476.20,45879.12,0,0.32,6690.6,6684.1,6709.2,6669.2,304870000.0,0.10
2,20200103,317.12,315.24,318.21,314.64,0,0.66,3217.07,3230.48,3231.26,...,46217.85,45904.82,0,-0.19,6733.5,6690.6,6788.2,6690.6,365160000.0,0.64
3,20200106,317.66,317.41,319.35,317.38,0,0.17,3202.74,3214.60,3214.92,...,46062.83,44658.76,0,-3.21,6735.7,6733.1,6738.6,6671.7,469750000.0,0.03
4,20200107,317.76,317.86,318.20,316.65,0,0.03,3189.50,3202.49,3220.35,...,45268.74,44412.05,0,-0.38,6826.4,6735.7,6834.3,6735.7,447410000.0,1.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,20221123,364.04,368.37,369.07,363.67,0,-1.18,3245.90,3250.56,3265.36,...,45883.07,44337.84,0,-0.89,7231.8,7181.3,7246.6,7181.3,575730000.0,0.70
756,20221124,361.10,364.04,364.81,360.58,0,-0.81,3256.29,3247.26,3269.79,...,45368.18,44601.12,0,0.77,7241.8,7231.8,7264.6,7231.8,592120000.0,0.14
757,20221125,363.63,361.10,363.87,359.26,0,0.70,3267.27,3256.79,3271.46,...,45849.74,44946.29,0,0.97,7259.5,7241.8,7268.5,7239.4,552060000.0,0.24
758,20221128,365.13,363.61,365.45,362.05,0,0.41,3208.77,3266.79,3266.79,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000.0,-0.42


In [ ]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
stock_table_list = ['035720'] # '035720' '035420'
complete_df = get_sqlalchemy_stock_data(sqlalchemy_conn, stock_table_list, investing_df)

  0%|          | 0/1 [00:00<?, ?it/s]

035720


100%|██████████| 1/1 [03:17<00:00, 197.32s/it]


In [ ]:
send_df = complete_df.dropna().copy()

In [ ]:
send_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,20201029,901,68645.0,68746.0,68344.0,68344.0,88652.0,6.081000e+09,84219.0,4433.0,...,32335.88,31897.50,0,-0.18,5960.3,6057.7,6057.7,5940.0,700200000.0,-1.61
1,20201029,902,68444.0,68746.0,68344.0,68746.0,31243.0,2.142870e+09,112813.0,7082.0,...,32335.88,31897.50,0,-0.18,5960.3,6057.7,6057.7,5940.0,700200000.0,-1.61
2,20201029,903,68746.0,69047.0,68645.0,68946.0,24647.0,1.697320e+09,135724.0,8818.0,...,32335.88,31897.50,0,-0.18,5960.3,6057.7,6057.7,5940.0,700200000.0,-1.61
3,20201029,904,68946.0,69247.0,68846.0,69147.0,24029.0,1.659230e+09,158090.0,10481.0,...,32335.88,31897.50,0,-0.18,5960.3,6057.7,6057.7,5940.0,700200000.0,-1.61
4,20201029,905,69147.0,69348.0,68946.0,68946.0,33062.0,2.286990e+09,187637.0,13996.0,...,32335.88,31897.50,0,-0.18,5960.3,6057.7,6057.7,5940.0,700200000.0,-1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196090,20221128,1517,54500.0,54700.0,54500.0,54600.0,4249.0,2.319900e+08,480431.0,787977.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000.0,-0.42
196091,20221128,1518,54600.0,54700.0,54600.0,54600.0,2614.0,1.427800e+08,480978.0,790044.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000.0,-0.42
196092,20221128,1519,54600.0,54700.0,54600.0,54700.0,3511.0,1.917200e+08,481137.0,793396.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000.0,-0.42
196093,20221128,1520,54600.0,54800.0,54600.0,54700.0,10159.0,5.553400e+08,487196.0,797496.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000.0,-0.42


In [ ]:
date_df = send_df['날짜'].copy()
corr_matrix = send_df.corr()
cor = corr_matrix["pct_label"]


39

In [ ]:
cor1 = cor[(cor.values>0.12) | (cor.values<-0.12)
        | (cor.index == '고가') | (cor.index == '시가') | (cor.index == '종가') | (cor.index == '저가')]

cor1 = cor1.drop('pct_label')  # 고가, 종가, 시가, 저가
len(cor1)
# cor1.to_pickle('./pickle/6개월_10개_cor_0.12_강화학습.pkl')

22

In [ ]:
cor1

시간                 -0.320369
시가                 -0.060011
고가                 -0.059891
저가                 -0.060324
종가                 -0.059864
거래량                 0.125637
전일대비                0.305598
기관순매수량              0.203761
INRKRW내역_변동        -0.133868
SGDKRW내역_변동        -0.129061
독일5년채권수익율내역_변동      0.133342
스위스1개월채권수익율내역_변동    0.131081
코스닥내역_변동            0.142774
코스피50내역_거래량         0.174997
코스피50내역_변동          0.169811
코스피지수내역_변동          0.155717
터키3년채권수익율내역_종가      0.120163
터키3년채권수익율내역_오픈      0.120163
터키3년채권수익율내역_고가      0.120167
터키3년채권수익율내역_저가      0.120123
한국2년채권수익율내역_변동     -0.135510
항셍내역_변동             0.143627
Name: pct_label, dtype: float64

In [ ]:
send_df

,시간,시가,고가,저가,종가,거래량,전일대비,기관순매수량,AEDKRW내역_변동,CADKRW내역_변동,...,코스피50내역_거래량,코스피50내역_변동,코스피지수내역_변동,터키3년채권수익율내역_종가,터키3년채권수익율내역_오픈,터키3년채권수익율내역_고가,터키3년채권수익율내역_저가,한국2년채권수익율내역_변동,항셍내역_변동,호주SPASX내역_거래량
날짜,,,,,,,,,,,,,,,,,,,,,
20201029,901,68645.0,68746.0,68344.0,68344.0,88652.0,-703,-150166,-0.42,-0.47,...,63810000.0,-0.98,-0.79,14.52,14.10,14.52,14.100,0.00,-0.49,700200000.0
20201029,902,68444.0,68746.0,68344.0,68746.0,31243.0,-703,-150166,-0.42,-0.47,...,63810000.0,-0.98,-0.79,14.52,14.10,14.52,14.100,0.00,-0.49,700200000.0
20201029,903,68746.0,69047.0,68645.0,68946.0,24647.0,-703,-150166,-0.42,-0.47,...,63810000.0,-0.98,-0.79,14.52,14.10,14.52,14.100,0.00,-0.49,700200000.0
20201029,904,68946.0,69247.0,68846.0,69147.0,24029.0,-703,-150166,-0.42,-0.47,...,63810000.0,-0.98,-0.79,14.52,14.10,14.52,14.100,0.00,-0.49,700200000.0
20201029,905,69147.0,69348.0,68946.0,68946.0,33062.0,-703,-150166,-0.42,-0.47,...,63810000.0,-0.98,-0.79,14.52,14.10,14.52,14.100,0.00,-0.49,700200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221128,1517,54500.0,54700.0,54500.0,54600.0,4249.0,-1500,-238434,0.41,-0.34,...,35000000.0,-1.18,-1.21,9.30,10.37,10.37,8.835,1.14,-1.57,700990000.0
20221128,1518,54600.0,54700.0,54600.0,54600.0,2614.0,-1500,-238434,0.41,-0.34,...,35000000.0,-1.18,-1.21,9.30,10.37,10.37,8.835,1.14,-1.57,700990000.0
20221128,1519,54600.0,54700.0,54600.0,54700.0,3511.0,-1500,-238434,0.41,-0.34,...,35000000.0,-1.18,-1.21,9.30,10.37,10.37,8.835,1.14,-1.57,700990000.0


In [ ]:
col_list = list(cor1.index)
# send_df = send_df.drop(['pct_label'],axis=1)
# send_df = send_df.set_index(['날짜'])
send_df = send_df[col_list]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
complete_df = min_max_scaler.fit_transform(send_df)

In [ ]:
complete_df = pd.DataFrame(complete_df, columns=col_list, index=date_df.values)
complete_df

,시간,시가,고가,저가,종가,거래량,전일대비,기관순매수량,INRKRW내역_변동,SGDKRW내역_변동,...,코스닥내역_변동,코스피50내역_거래량,코스피50내역_변동,코스피지수내역_변동,터키3년채권수익율내역_종가,터키3년채권수익율내역_오픈,터키3년채권수익율내역_고가,터키3년채권수익율내역_저가,한국2년채권수익율내역_변동,항셍내역_변동
20201029,0.000000,0.175797,0.174879,0.174752,0.173397,0.031428,0.569115,0.619084,0.465995,0.391549,...,0.597030,0.259438,0.288000,0.364486,0.0007,0.0005,0.000557,0.000706,0.699577,0.380181
20201029,0.001372,0.174195,0.174879,0.174752,0.176603,0.011076,0.569115,0.619084,0.465995,0.391549,...,0.597030,0.259438,0.288000,0.364486,0.0007,0.0005,0.000557,0.000706,0.699577,0.380181
20201029,0.002743,0.176603,0.177262,0.177160,0.178198,0.008738,0.569115,0.619084,0.465995,0.391549,...,0.597030,0.259438,0.288000,0.364486,0.0007,0.0005,0.000557,0.000706,0.699577,0.380181
20201029,0.004115,0.178198,0.178844,0.178768,0.179801,0.008519,0.569115,0.619084,0.465995,0.391549,...,0.597030,0.259438,0.288000,0.364486,0.0007,0.0005,0.000557,0.000706,0.699577,0.380181
20201029,0.005487,0.179801,0.179644,0.179568,0.178198,0.011721,0.569115,0.619084,0.465995,0.391549,...,0.597030,0.259438,0.288000,0.364486,0.0007,0.0005,0.000557,0.000706,0.699577,0.380181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221128,0.844993,0.062998,0.063712,0.064000,0.063796,0.001506,0.538462,0.593226,0.780856,0.616901,...,0.291089,0.142222,0.265143,0.308411,0.0000,0.0000,0.000000,0.000000,0.860367,0.310233
20221128,0.846365,0.063796,0.063712,0.064800,0.063796,0.000927,0.538462,0.593226,0.780856,0.616901,...,0.291089,0.142222,0.265143,0.308411,0.0000,0.0000,0.000000,0.000000,0.860367,0.310233
20221128,0.847737,0.063796,0.063712,0.064800,0.064593,0.001245,0.538462,0.593226,0.780856,0.616901,...,0.291089,0.142222,0.265143,0.308411,0.0000,0.0000,0.000000,0.000000,0.860367,0.310233
20221128,0.849108,0.063796,0.064503,0.064800,0.064593,0.003601,0.538462,0.593226,0.780856,0.616901,...,0.291089,0.142222,0.265143,0.308411,0.0000,0.0000,0.000000,0.000000,0.860367,0.310233


In [70]:
complete_df.to_csv('./kakao_035720.csv', encoding='utf-8-sig')

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15751963213133421779
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3667263488
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4189665415739355941
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]